In [50]:
import pandas as pd
import yaml
from pytwitter import Api


def process_yaml():
    with open("config.yaml") as file:
        return yaml.safe_load(file)

config = process_yaml()
api = Api(bearer_token=config['twitter_api_v2']['bearer_token'], sleep_on_rate_limit=True)

df =  pd.read_csv('esg_twitter.csv')
df.head(1)

,Unnamed: 0,Ticker,username,Name,Industry,Total ESG Risk Score,Environment Risk Score,Social Risk Score,Governance Risk Score,user_id
0,0,MMM,3m,3M Company (MMM),Industrials,34,12.6,12.6,8.4,378197959


In [60]:
# https://pypi.org/project/python-twitter-v2/

tweets_df = pd.DataFrame(columns=['username', 'user_id', 'text'])
for index, username, user_id in df[['username', 'user_id']][260:].itertuples():
    try:
        tweets = []
        next_token = None
        while next_token or len(tweets) == 0:
            res = api.get_timelines(user_id=user_id,
                                    max_results=100,
                                    pagination_token=next_token,
                                    start_time='2019-01-01T00:00:00Z')
            tweets.extend(res.data)
            next_token = res.meta.next_token

        print(user_id, username, len(tweets))
        texts = [{'username': username, 'user_id': user_id, 'text': tweet.text} for tweet in tweets]
        tweets_df = tweets_df.append(texts, ignore_index=True, sort=False)
        tweets_df.to_csv('260-.csv')
    except:
        print("ERROR no tweets?",index, username, user_id)

18639734 nasdaq 3250
2588219916 novglobal 404
15897465 netapp 3139
121580581 newell_brands 319
437789543 newmontcorp 1158
1516521758 newscorp 58
94190179 nexteraenergy 82
ERROR no tweets? 267 nielsenir 859654890
415859364 nike 1881
2329361226 nisourceinc 169
66472414 nscorp 891
308506248 northerntrust 857
1173980018009001984 nortonlifelock 554
17045115 cruisenorwegian 3248
32880255 nrgenergy 424
ERROR no tweets? 275 nucorcorp 361259570
61559439 nvidia 1032
32516857 oreillyauto 558
42641711 weareoxy 290
64834590 oneok 51
809273 oracle 3246
2645632766 packagingcorp 856
112423234 parkerhannifin 3054
39750799 paychex 2884
30018058 paypal 1165
3070701243 pentair 550
209116528 peoplesunited 1712
21346619 pepsico 2063
ERROR no tweets? 288 perkinelmer 128564900
1430592241 perrigocompany 484
56488059 pfizer 1973
914311278 insidepmi 3249
405085229 phillips66co 564
979244048 pxdtweets 77
40059448 pncbank 1204
164365554 principal 1127
3331681 progressive 1902
108336372 prologis 858
1194947442 prud

In [55]:
# tweets_df.to_csv('258.csv')